# Topic Modeling and Gibbs Sampling

Задача: описать текст через распределение весов по некоторому фиксированному набору топиков (тегов). Например, для набора тегов Политика, Военные сражения, Спорт, Интернет, Драма представить роман "Война и мир" как вектор (0.3, 0.2, 0, 0, 0.5), а статью в газете про допинг в велоспорте как вектор (0.1, 0, 0.7, 0, 0.2).

Для чего, например, это нужно: имея векторное представление для текстов, тексты можно сравнивать, рекомендовать похожие.

Условие: даны только набор текстов и количество тем.

## Немного теории

Будем представлять текст как неупорядоченный набор слов (Bag-of-words model). Предположим, что имеется K тегов и для каждого тега выбрано распределение $\phi_k$ над списком всевозможных слов (словарем из N слов). По сути, каждое $\phi_k$ - это вектор длины N из неотрицательных величин, в сумме дающих 1. Вектора $\phi_k$ независимы и моделируются распредеделением Дирихле $Dir(\beta)$. Теперь, чтобы собрать текст d из $n$ слов, будем действовать по следующей схеме:


* выберем распредление для тегов $\theta_d$. Вновь, $\theta_d$ - это вектор длины K из неотрицательных величин, в сумме дающих 1. Поэтому естественно брать $\theta_d \sim Dir(\theta | \alpha)$

* Для i от 1 до n:
  * выберем тег $z_i$ согласно распределению $\theta_d$
  * выберем слово $w_i$ из распределения для данного тега, т.е. $w_i \sim \phi_{z_i}$
  * добавляем слово $w_i$ в текст.

Полученная модель называется моделью LDA (Latent Dirichlet Allocation). Описанная  схема задает совместное распределение скрытых и наблюдаемых параметров по всем текстам корпуса размера M в виде:


$p(\textbf{w}, \textbf{z}, \theta, \phi | \alpha, \beta) = Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)$.

Здесь $\textbf{w}$ и $\textbf{z}$ обозначают вектора слов и тегов по всем текстам, $\theta$ - набор из $\theta_d$ для каждого документа (матрица $M\times K$), $\phi$ - набор из $\phi_k$ для каждого тега (матрица $K\times N$).


![img](https://www.researchgate.net/publication/336065245/figure/fig1/AS:807371718815752@1569503826964/Latent-Dirichlet-allocation-LDA-process-and-its-two-outputs-a-LDA-document.ppm)

Наша задача - восстановить распределение $p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)$.


Немного упростим жизнь, и поставим себе задачей восстановить распределение $p(\textbf{z} | \textbf{w}, \alpha, \beta) = \int\int p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)\textrm{d}\theta\textrm{d}\phi$.

В этот момент на помощь приходить алгоритм Gibbs Sampling. Напомним, для оценки набора парамеров $\textbf{z} = (z_1, z_2, ..., z_m)$ используется схема:

$z_i^{(t)} \sim p(z_i^{(t)}\ | \ z_1=z_1^{t}, ..., z_{i-1}=z_{i-1}^{t},
z_{i+1}=z_{i+1}^{t-1}, z_{m}=z_{m}^{t-1})$.

Условные распределения выводятся так. Сначала замечаем, что

$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) = \frac{p(z_i,\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}  = \frac{p(\textbf{z}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}_{\hat{i}}| \alpha, \beta)  p(w_i|\alpha, \beta)}$.

Здесь $\textbf{z}_{\hat{i}}$, $\textbf{w}_{\hat{i}}$ - вектора без $i$-oй копмоненты. 

Далее расписываем:

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \int\int p(\textbf{z}, \textbf{w}, \theta, \phi| \alpha, \beta)\textrm{d}\theta\textrm{d}\phi = \int\int Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)\textrm{d}\theta\textrm{d}\phi = 
\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta \int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi$

и обнаруживаем, что оба интеграла в последнем выражении вычисляются аналитически. Для примера первый:

$\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta = \prod\limits_d \int Dir(\theta_d | \alpha) Cat(\textbf{z}_d|\theta_d)\textrm{d}\theta_d = \prod\limits_d \int \frac{1}{B(\alpha)}\prod\limits_k \theta_{d, k}^{\alpha-1}\prod\limits_i \theta_{d, z_i}\textrm{d}\theta_d = \prod\limits_d\frac{1}{B(\alpha)}\int\prod\limits_k \theta_{d, k}^{n_{d, k} + \alpha - 1}\textrm{d}\theta_d = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)}$.

Здесь  $n_{d,k}$ - количество тэгов $k$ в тексте $d$, $n_{d,\cdot}$ - вектор длины $K$ из этих величин.

Аналогично, второй интеграл 
$\int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi = \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$,

где $n_{k,\cdot}$ - вектор длины $N$ встречаемости слов внутри тэга $k$.

Получаем: 

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$.

Теперь
$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(n_{d,\cdot}^{\hat{i}} + \alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(n_{k,\cdot}^{\hat{i}} + \beta)}$.

Знак $\propto$ означает пропорциональность с точностью до общего множителя $p(w_i|\alpha, \beta)$. Векторы $n_{d,\cdot}^{\hat{i}}$ и $n_{k,\cdot}^{\hat{i}}$ получены из векторов $n_{d,\cdot}$ и $n_{k,\cdot}$ после выбрасывания $z_i$. 

Выражение упрощается дальше, расписывая бета-функцию через гамма-функции. Напомним,
$B(x_1, ..., x_m) = \frac{\Gamma(x_1)\cdot...\cdot\Gamma(x_m)}{\Gamma(x_1 + ... + x_m)}$, а также $\Gamma(n) = (n-1)\Gamma(n-1)$. Получим:

$p(z_i=k |\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto (n_{d_i, k}^{\hat{i}} + \alpha_k) \frac{n_{k, w_i}^{\hat{i}} + \beta_{w_i}}{\sum\limits_{w}(n_{k, w}^{\hat{i}} + \beta_{w})}$.


С этого места можно полностью собрать алгоритм моделирования плотности $p(\textbf{z}| \textbf{w}, \alpha, \beta)$. Введем  обозначение $n_k$ - количество слов, отнесенных к тегу $k$, $W$ - общее количество слов в корпусе, $\beta_{sum} = \sum\limits_w\beta_w$

Алгоритм:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * для каждого $k$ от 1 до $K$:
        * $I = I\{z_i = k\}$ (индикатор)
        * вычисляем $p_k = (n_{d_i, k} + \alpha_k - I) \frac{n_{k, w_i} + \beta_{w_i} - I}{n_k + \beta_{sum} - I}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * обновляем счетчики для учета обновленого значения $z_i$

На практике удобно реализовавать так:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$, заполненные нулями
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * $n_{d_i, z_i} \mathrel{-}= 1$, $n_{z_i, w_i} \mathrel{-}= 1$, $n_{z_i} \mathrel{-}= 1$
      * для каждого $k$ от 1 до $K$:
        * вычисляем $p_k = (n_{d, k} + \alpha_k) \frac{n_{k, w_i} + \beta_{w_i}}{n_k + \beta_{sum}}$
      * сэмплим новый $z_i$ из полученного распределения $(p_1, ..., p_K)$
      * $n_{d_i, z_i} \mathrel{+}= 1$, $n_{z_i, w_i} \mathrel{+}= 1$, $n_{z_i} \mathrel{+}= 1$





Восстановив распредление для $\textbf{z}$, можем оценить $\theta$ и $\phi$, о которых мы ненадолго забыли. Оценить можно, например, через матожидание по апостериорным распределениям. Получите формулы самостоятельно!

Литература:

http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

https://www.cs.cmu.edu/~mgormley/courses/10701-f16/slides/lecture20-topic-models.pdf

Перейдем к практике.

## Датасет

Возьмем популярный датасет [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/), встроенный в пакет ```sklearn```. Датасет состоит из ~20К текстов, классифицированных на 20 категорий. Датасет разбит на ```train``` и ```test```. Для загрузки используем  модуль ```fetch_20newsgroups```, в параметрах указать, что мета информацию о тексте загружать не нужно:

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Выведем список категорий текстов:

In [2]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Атрибут ```traget``` хранит номера категорий для текстов из обучающей выборки:

In [3]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Доступ к самим текстам через атрибут ```data```. Выведем текст и категорию случайного примера из обучающего датасета:

In [4]:
n = 854
print('Topic = {0}\n'.format(newsgroups_train.target_names[newsgroups_train.target[n]]))
print(newsgroups_train.data[n])

Topic = rec.motorcycles

hey... I'm pretty new to the wonderful world of motorcycles... I just
bought
a used 81 Kaw KZ650 CSR from a friend.... I was just wondering what kind of

saddle bags I could get for it (since I know nothing about them)  are there
bags for the gas tank?  how much would some cost, and how much do they
hold?
thanks for your advice!!!  I may be new to riding, but I love it
already!!!!
:)




## Векторное представление текста

Представим текст как вектор индикаторов вхождений слов из некоторого словаря в текст. Это простейшая модель BOF. 

Сформируем словарь на основе нашего набора текстов. Для этого используем модуль ```CountVectorizer```:

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word',min_df = 0.0021)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0.0021,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'should', 'yours', 'both', 'everything', 'sometime', 'were', 'made', 'throughout', 'nobody', 'beyond', 'get', 'their', 'ever', 'ten', 'onto', 'our', 'some', 'from', 'five', 'forty', 'when', 'six', 'anyone', 'fire', 'top', 'name', 'do', 'during', 'while', 'her', 'off', 'whatever...ribe', 'via', 'never', 'something', 'former', 'except', 'without', 'amoungst', 'ie', 'cant', 'out'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Количество проиндексированных слов:

In [4]:
len(vectorizer.vocabulary_)

5162

Проиндексированные слова и их индексы:

In [5]:
vectorizer.vocabulary_

{'wondering': 5099,
 'car': 857,
 'saw': 4084,
 'day': 1335,
 'door': 1557,
 'sports': 4403,
 'looked': 2792,
 'late': 2661,
 'early': 1607,
 'called': 829,
 'doors': 1558,
 'really': 3800,
 'small': 4315,
 'addition': 310,
 'separate': 4177,
 'rest': 3951,
 'body': 727,
 'know': 2630,
 'model': 3016,
 'engine': 1688,
 'specs': 4383,
 'years': 5149,
 'production': 3641,
 'history': 2264,
 'info': 2421,
 'looking': 2793,
 'mail': 2834,
 'fair': 1844,
 'number': 3188,
 'souls': 4355,
 'upgraded': 4882,
 'si': 4253,
 'clock': 1004,
 'shared': 4223,
 'experiences': 1795,
 'send': 4167,
 'brief': 765,
 'message': 2947,
 'procedure': 3627,
 'speed': 4387,
 'cpu': 1255,
 'rated': 3771,
 'add': 307,
 'cards': 859,
 'heat': 2231,
 'hour': 2305,
 'usage': 4890,
 'floppy': 1958,
 'disk': 1513,
 'functionality': 2038,
 '800': 224,
 'floppies': 1957,
 'especially': 1727,
 'requested': 3920,
 'days': 1336,
 'network': 3140,
 'knowledge': 2632,
 'base': 624,
 'upgrade': 4881,
 'haven': 2213,
 'answer

Индекс, например, для слова anyone:

In [7]:
vectorizer.vocabulary_.get('car')

857

А теперь преобразуем строку в вектор:

Какой тип имеет объект, на который указывает ```x```?

In [10]:
type(x)

scipy.sparse.csr.csr_matrix

Разреженная матрица!

### Отступление про разреженные матрицы

Список ненулевых элементов матрицы:

In [11]:
x.data

array([2, 1, 1], dtype=int64)

Индексы строк и столбцов для ненулевых элементов:

In [12]:
x.nonzero()

(array([0, 0, 0], dtype=int32), array([ 889, 4243, 5293], dtype=int32))

Преобразование к объекту ndarray (именно после приведения к такому виду разреженные матрицы можно подставлять в функции, например, библиотеки Numpy):

In [13]:
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

Вернемся к словарю. Раскодируем вектор ```x``` в список слов:

In [14]:
vectorizer.inverse_transform(x)

[array(['car', 'saw', 'wondering'], dtype='<U15')]

Пропало слово ```I```. Но дело в том, что по умолчанию ```CountVectorizer``` отбрасывает последовательности, короче 2 символов. На это указывает параметр ```token_pattern='(?u)\\b\\w\\w+\\b'```.

Переведем весь набор текстов обучающего датасета в набор векторов, получим матрицу ```X_train```:

In [8]:
X_train = vectorizer.fit_transform(newsgroups_train.data)

In [9]:
u = X_train.toarray()

In [10]:

v = []
for k in range(len(u)):
    w = []
    t = u[k].nonzero()[0]
    for i in range(len(t)):
        for j in range(u[k][t[i]]):
            w.append(t[i])
    v.append(w)

In [11]:
u[1234].nonzero()

(array([ 302,  312,  495,  563,  625,  631,  677,  872,  911,  995,  996,
        1055, 1056, 1143, 1187, 1250, 1272, 1274, 1275, 1475, 1546, 1547,
        1555, 1611, 1638, 1642, 1757, 1829, 1833, 2170, 2171, 2347, 2378,
        2521, 2526, 2589, 2626, 2627, 2693, 2710, 2741, 2769, 2791, 2812,
        2901, 2919, 3085, 3107, 3122, 3268, 3304, 3533, 3567, 3590, 3602,
        3625, 3769, 3770, 3783, 3827, 3829, 3845, 3877, 3903, 3995, 4159,
        4237, 4250, 4427, 4436, 4576, 4615, 4707, 4708, 4709, 4780, 4814,
        4891, 4899, 4902, 4952, 5006, 5015, 5036, 5037], dtype=int32),)

In [12]:
listt = np.array(v)

In [13]:
listt[0][0]

310

In [14]:
listt

array([list([310, 727, 829, 857, 857, 857, 857, 1335, 1557, 1558, 1607, 1688, 2264, 2421, 2630, 2661, 2792, 2793, 2834, 3016, 3641, 3800, 3951, 4084, 4177, 4315, 4383, 4403, 5099, 5149]),
       list([224, 307, 307, 433, 624, 765, 859, 1004, 1004, 1255, 1335, 1336, 1513, 1727, 1795, 1795, 1844, 1957, 1958, 2038, 2213, 2231, 2305, 2632, 2947, 3140, 3188, 3627, 3771, 3920, 4167, 4223, 4253, 4355, 4387, 4387, 4683, 4881, 4882, 4890]),
       list([31, 57, 57, 62, 62, 62, 62, 84, 275, 296, 302, 331, 432, 438, 438, 438, 439, 680, 698, 798, 798, 1098, 1221, 1311, 1335, 1335, 1492, 1513, 1513, 1515, 1515, 1515, 1546, 1546, 1555, 1555, 1588, 1599, 1631, 1651, 1659, 1690, 1789, 1892, 1911, 1920, 1921, 1966, 2060, 2114, 2117, 2140, 2213, 2227, 2227, 2227, 2245, 2265, 2293, 2381, 2421, 2421, 2470, 2589, 2589, 2630, 2734, 2739, 2749, 2771, 2793, 2794, 2822, 2823, 2823, 2824, 2846, 2878, 2906, 3031, 3144, 3147, 3247, 3263, 3263, 3390, 3396, 3436, 3482, 3495, 3534, 3548, 3548, 3548, 3596, 3623, 3683

О пользе разреженных матриц. Отношение числа ненулевых элементов ко всем элементам матрицы ```X_train```:

In [15]:
X_train.nnz / np.prod(X_train.shape)

0.008769997391224006


Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы. 

In [23]:
K = 20
N = 101322
l = 11314
z = np.random.randint(0,20,N)
nd = np.zeros((l,K))
n = np.zeros(K)
nk = np.zeros((K,N))
phi = np.zeros((K,N))
phi = phi + 1/N
tetta = np.zeros((K,l))
tetta = tetta + 1/K
print(z)
print(len(z))

[ 6 10  1 ...  3  7 14]
101322


In [16]:
len(u)

11314

In [17]:
def randomInit(ndz,nzw,nz,Z,l):
    for d, doc in enumerate(l):
        Cur = []
        for w in doc:
            pz = np.divide(np.multiply(ndz[d, :], nzw[:, w]), nz)
            z = np.random.multinomial(1, pz / pz.sum()).argmax()
            Cur.append(z)
            ndz[d, z] += 1
            nzw[z, w] += 1
            nz[z] += 1
        Z.append(Cur)

In [18]:
def sampling(ndz,nzw,nz,Z,l):
    for d, doc in enumerate(l):
        for index, w in enumerate(doc):
            z = Z[d][index]
            ndz[d, z] -= 1
            nzw[z, w] -= 1
            nz[z] -= 1
            pz = np.divide(np.multiply(ndz[d, :], nzw[:, w]), nz)
            z = np.random.multinomial(1, pz / pz.sum()).argmax()
            Z[d][index] = z 
            ndz[d, z] += 1
            nzw[z, w] += 1
            nz[z] += 1

In [19]:
def getwords(diction, value):
    words = list()
    items = diction.items()
    for item  in items:
        if item[1] == value:
            words.append(item[0])
    return  words

In [20]:
N_w = 5162
alpha = 5
beta = 0.1
iterations = 50
Z = []
K = 20

N = len(listt) 
M = N_w 
ndz = np.zeros([N, K]) + alpha
nzw = np.zeros([K, M]) + beta
nz = np.zeros([K]) + M * beta

randomInit(ndz,nzw,nz,Z,listt)

for i in range(0, iterations):
    sampling(ndz,nzw,nz,Z,listt)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [27]:
topics = []  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!
maxNum = 10
for z in range(0, K):
    print(z)
    ids = nzw[z, :].argsort()
    topic = []
    for j in ids:
        topic.insert(0, getwords(vectorizer.vocabulary_, j))
    topics.append(topic[ : min(10, len(topic))])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [28]:
for n in range (20):
    print(n, topics[n])
    print('\n')
newsgroups_train.target_names

0 [['time'], ['little'], ['does'], ['really'], ['ll'], ['trying'], ['problem'], ['try'], ['thing'], ['point']]


1 [['don'], ['just'], ['know'], ['need'], ['want'], ['doesn'], ['probably'], ['work'], ['actually'], ['like']]


2 [['people'], ['right'], ['say'], ['law'], ['state'], ['think'], ['person'], ['fact'], ['case'], ['rights']]


3 [['windows'], ['version'], ['window'], ['image'], ['dos'], ['graphics'], ['color'], ['display'], ['use'], ['using']]


4 [['mr'], ['gun'], ['president'], ['000'], ['states'], ['american'], ['control'], ['united'], ['guns'], ['health']]


5 [['key'], ['government'], ['chip'], ['use'], ['encryption'], ['public'], ['security'], ['new'], ['bit'], ['keys']]


6 [['drive'], ['card'], ['car'], ['disk'], ['scsi'], ['hard'], ['bit'], ['speed'], ['driver'], ['memory']]


7 [['year'], ['got'], ['better'], ['good'], ['best'], ['won'], ['game'], ['just'], ['lost'], ['pretty']]


8 [['use'], ['does'], ['used'], ['case'], ['example'], ['possible'], ['output'], ['note

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

По полученным наборам слов заметно, что алгоритм работает в целом так, как надо, например, тема 11 скорее всего атеизм, 6 связана с техникой, 9 с хоккеем.